In [ ]:
!pip3 install google-cloud-speech==2.11.0

     |████████████████████████████████| 122 kB 5.8 MB/s 
     |████████████████████████████████| 517 kB 29.4 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
     |████████████████████████████████| 1.1 MB 29.6 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="gc.json"

In [ ]:
from scipy.io import wavfile
import scipy.signal as sps
import numpy as np
import pandas as pd
from pydub import AudioSegment

In [ ]:
#   #Read file
# sampling_rate, data = wavfile.read("1-phrase.wav")
# print(sampling_rate)
#   # Resample data
# data = sps.resample(data, 44100)

# content = np.ndarray.tobytes(data)
# print(content)

50000
b'T\xc3\x96\xb0\x1eU5@m)o\'\xcd\xb0F@\xa4K1\xa3I\xf0B@\x13\xc1\xc2\x8a,`B@\x0b\xeb\xb3\xdc\xbcIA@\x12\xb0\x83\xba\x15\xdb7@\xcd\xb5\r\xeb\x9b\x82,@\xde\x00v\xf4\x04\x81\x16@ \x85\x19\xc4s\xb28@\x0f\xfc\x8bS\xcf\xa7;@\xad\xe2\x1b \x82\xf15@\xccZ4\xb6\xcc\xbc0@\xaa\x00\xee\xda)\xe8A@}\xfc\xe2\xc5\x95YA@p9O\xa2\x98_>@\x81\x8f\xfd\xdf\x93\x1dA@R\x17\xf7\xd4\x81\xe0@@\x84\x1bB\xe2\x89\x80C@\x80\xa4\x95y\xe4\x99<@W\xf38i\x07\x9fA@gQ&\xa7\x80\xd3D@\x8e\x19\xe5\x86U\xd7E@I>\x81\xf8\x80L@@\x02\x04\x9e\x85\xad\xd9?@v\x94"\xd6\xde0C@\xcf\xc6\x08\x93\x9eg7@JQ\x83\xa1\xb8`?@B\xb5\x07\x19\x95\xc8@@[\x1b\xf0\xf8m\xe7A@\x1b\xbc\x08\x9f\xff\xae<@\xa21\xa7"3\xd6E@n/\x88L\xae\x9fF@\xa0\xfb\xb6*\x98\x08A@\x9e\x89\xe8\\VvD@\xe2t\xfe\xb4:/I@l\xe5\x149\x9f\x08B@\xf6\x8e\xd7\xd7\xbbpE@\x14\xfa\x12\xe1\xf3\xdfE@\x7f~\xca\x93\xb2\xffG@\x07\xa2\x9f&\xf9\xe7F@\xf8\xf6\x9d\x9d\x0f`I@\xe5\x81X\x8a\x1f\x9fG@\xe0\x19S/\xd8\xe8C@\xea\xb2\xb6J\xa6\tK@\xd7"0R\x89|F@\xc4^\x97\xe8\xba\x00C@\xc6\x17\xdc\xc3\xceZJ@\xf

In [ ]:
def transcribe_file(speech_file):
  from google.cloud import speech
  import io

  client = speech.SpeechClient()

  sampling_rate, data = wavfile.read(speech_file)
  last = np.ndarray.tobytes(data)

  audio = speech.RecognitionAudio(content=last)


  config = speech.RecognitionConfig(
      encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
      sample_rate_hertz=44100,
      language_code="de-DE",
      enable_word_time_offsets=True,
  )

  operation = client.long_running_recognize(config=config, audio=audio)


  result = operation.result(timeout=90)


  transcript = ""

  for result in result.results:
      alternative = result.alternatives[0]
        #print("Transcript: {}".format(alternative.transcript))
        #print("Confidence: {}".format(alternative.confidence))

      for word_info in alternative.words:
          word = word_info.word
          start_time = word_info.start_time
          end_time = word_info.end_time

          print(
              f"Word: {word}, start_time: {start_time.total_seconds()}, end_time: {end_time.total_seconds()}"
          )
          transcript = transcript+ f'{word}, {start_time.total_seconds()}, {end_time.total_seconds()}' + '\n'

      
  return transcript  

In [ ]:
def write_transcripts(transcript_file, transcript):
  f = open(transcript_file,"w",encoding="utf-8")
  f.write(transcript)
  f.close()
  return

In [ ]:
def get_GUTEN(file_csv, file_wav):
  df = pd.read_csv(file_csv, header = None, names = ["name", "start", "end"])
  t1=df.iat[0,1]*1000
  t2=df.iat[0,2]*1000
  newAudio = AudioSegment.from_wav(file_wav)
  newAudio = newAudio[t1:t2]
  exp = "zz_guten" + file_csv.split('.')[0]+ ".wav"
  newAudio.export(exp, format="wav") #Exports to a wav file in the current path.

In [ ]:
def get_MORGEN(file_csv, file_wav):
  df = pd.read_csv(file_csv, header = None, names = ["name", "start", "end"])
  t1=df.iat[1,1]*1000
  t2=df.iat[1,2]*1000
  newAudio = AudioSegment.from_wav(file_wav)
  newAudio = newAudio[t1:t2]
  exp = "zz_morgen" + file_csv.split('.')[0]+ ".wav"
  newAudio.export(exp, format="wav")

In [ ]:
def get_WIE(file_csv, file_wav):
  df = pd.read_csv(file_csv, header = None, names = ["name", "start", "end"])
  t1=df.iat[2,1]*1000
  t2=df.iat[2,2]*1000
  newAudio = AudioSegment.from_wav(file_wav)
  newAudio = newAudio[t1:t2]
  exp = "zz_wie" + file_csv.split('.')[0]+ ".wav"
  newAudio.export(exp, format="wav")

In [ ]:
def get_GEHT(file_csv, file_wav):
  df = pd.read_csv(file_csv, header = None, names = ["name", "start", "end"])
  t1=df.iat[3,1]*1000
  t2=df.iat[3,2]*1000
  newAudio = AudioSegment.from_wav(file_wav)
  newAudio = newAudio[t1:t2]
  exp = "zz_geht" + file_csv.split('.')[0]+ ".wav"
  newAudio.export(exp, format="wav")

In [ ]:
def get_ES(file_csv, file_wav):
  df = pd.read_csv(file_csv, header = None, names = ["name", "start", "end"])
  t1=df.iat[4,1]*1000
  t2=df.iat[4,2]*1000
  newAudio = AudioSegment.from_wav(file_wav)
  newAudio = newAudio[t1:t2]
  exp = "zz_es" + file_csv.split('.')[0]+ ".wav"
  newAudio.export(exp, format="wav")

In [ ]:
def get_IHNEN(file_csv, file_wav):
  df = pd.read_csv(file_csv, header = None, names = ["name", "start", "end"])
  t1=df.iat[5,1]*1000
  t2=df.iat[5,2]*1000
  newAudio = AudioSegment.from_wav(file_wav)
  newAudio = newAudio[t1:t2]
  exp = "zz_ihnen" + file_csv.split('.')[0]+ ".wav"
  newAudio.export(exp, format="wav")

In [ ]:
current_folder = "C:/Users/Dell/Desktop/INŻYNIERKA/PATHO WOMAN - 727/export/"

In [ ]:
for filename in os.listdir("/content/"):
  if filename.endswith(".wav"):
    x= transcribe_file(filename);
    transcript_file = filename.split('.')[0] + '.csv'
    write_transcripts(transcript_file, x)
    print(f'Transcript {transcript_file} created')

Word: guten, start_time: 0.0, end_time: 0.3
Word: Morgen, start_time: 0.3, end_time: 0.9
Word: wie, start_time: 0.9, end_time: 1.1
Word: geht, start_time: 1.1, end_time: 1.5
Word: es, start_time: 1.5, end_time: 1.5
Word: Ihnen, start_time: 1.5, end_time: 2.1
Transcript 2227-phrase.csv created
Word: guten, start_time: 0.0, end_time: 0.3
Word: Morgen, start_time: 0.3, end_time: 0.8
Word: wie, start_time: 0.8, end_time: 1.3
Word: geht, start_time: 1.3, end_time: 1.6
Word: es, start_time: 1.6, end_time: 1.8
Word: Ihnen, start_time: 1.8, end_time: 2.2
Transcript 2224-phrase.csv created
Word: guten, start_time: 0.0, end_time: 0.4
Word: Morgen, start_time: 0.4, end_time: 0.9
Word: wie, start_time: 0.9, end_time: 1.3
Word: geht, start_time: 1.3, end_time: 1.6
Word: es, start_time: 1.6, end_time: 1.7
Word: Ihnen, start_time: 1.7, end_time: 2.2
Transcript 2127-phrase.csv created
Word: guten, start_time: 0.0, end_time: 0.4
Word: Morgen, start_time: 0.4, end_time: 1.1
Word: wie, start_time: 1.1, e

In [ ]:
teścik1 = "105-phrase.wav"
y= transcribe_file(teścik1)
transcript_file = teścik1.split('.')[0] + '.csv'
write_transcripts(transcript_file, y)
print(f'Transcript {transcript_file} created')

Word: guten, start_time: 0.0, end_time: 0.5
Word: Morgen, start_time: 0.5, end_time: 1.1
Word: wie, start_time: 1.1, end_time: 1.6
Word: geht, start_time: 1.6, end_time: 2.4
Word: es, start_time: 2.4, end_time: 2.7
Word: Ihnen, start_time: 2.7, end_time: 3.7
Transcript 105-phrase.csv created


In [ ]:
for filename in os.listdir("/content/"):
  if filename.endswith(".csv"): #and filename.startswith("2"):
    audio = filename.split('.')[0]+ '.wav'
    get_GUTEN(filename, audio)
    get_MORGEN(filename, audio)
    get_WIE(filename, audio)
    get_GEHT(filename, audio)
    get_ES(filename, audio)
    get_IHNEN(filename, audio)

In [ ]:
get_GUTEN("100-phrase.csv", "100-phrase.wav")

In [ ]:
!zip -r 0-rest-PATHO-WOMAN.zip /content/ 

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.11.18/ (stored 0%)
  adding: content/.config/logs/2021.11.18/14.34.49.152157.log (deflated 91%)
  adding: content/.config/logs/2021.11.18/14.35.29.959577.log (deflated 86%)
  adding: content/.config/logs/2021.11.18/14.35.09.738473.log (deflated 53%)
  adding: content/.config/logs/2021.11.18/14.35.57.849518.log (deflated 54%)
  adding: content/.config/logs/2021.11.18/14.35.36.918574.log (deflated 53%)
  adding: content/.config/logs/2021.11.18/14.35.58.667492.log (deflated 53%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/

In [ ]:
!zip -r /content/plis.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.11.18/ (stored 0%)
  adding: content/.config/logs/2021.11.18/14.35.57.849518.log (deflated 54%)
  adding: content/.config/logs/2021.11.18/14.35.58.667492.log (deflated 53%)
  adding: content/.config/logs/2021.11.18/14.34.49.152157.log (deflated 91%)
  adding: content/.config/logs/2021.11.18/14.35.29.959577.log (deflated 86%)
  adding: content/.config/logs/2021.11.18/14.35.36.918574.log (deflated 53%)
  adding: content/.config/logs/2021.11.18/14.35.09.738473.log (deflated 53%)
  adding: content/.config/gce (stored 0%)
  adding: content/.confi